In [7]:
import mido
import numpy as np
import matplotlib.pyplot as plt
import PreProcessor as pp
from keras.models import Model
from tensorflow.keras.layers import MultiHeadAttention, Embedding, Dense, LayerNormalization, Dropout, Softmax, concatenate, Add
from keras import optimizers
import tensorflow as tf
from utils import *

In [3]:
dataset = pp.load_dataset("../adl-piano-midi")
ClassicSongs = pp.files_to_songs(dataset["Classical"])

channel_to_ind, ind_to_channel, note_to_ind, ind_to_note, velocity_to_ind, ind_to_velocity = pp.dicts_from_songs(ClassicSongs)
time_range = pp.ranges_from_songs(ClassicSongs)

n_Channels = len(channel_to_ind)
n_Notes = len(note_to_ind)
n_Velocities = len(velocity_to_ind)

print("\nNumber of channels   :",n_Channels,"\nNumber of notes      :",n_Notes,"\nNumber of velocities :",n_Velocities,"\nTime range           :",time_range[0],time_range[1])


Number of channels   : 12 
Number of notes      : 105 
Number of velocities : 128 
Time range           : 0.0 190.285


In [20]:
class MusicEmbedding(tf.keras.layers.Layer):
    def __init__(self, n_Channels, n_Notes, n_Velocities, d_model):
        super().__init__()
        self.d_model = d_model
        
        tot_dim = n_Channels + n_Notes + n_Velocities
        self.d_Channels = int((d_model-1)*n_Channels/tot_dim)
        self.d_Notes = int((d_model-1)*n_Notes/tot_dim)
        self.d_Velocities = int((d_model-1)*n_Velocities/tot_dim)
        while self.d_Channels + self.d_Notes + self.d_Velocities != d_model - 1 : self.d_Channels += 1
            
        self.Channel_Embedding = Embedding(n_Channels, self.d_Channels)
        self.Notes_Embedding = Embedding(n_Notes, self.d_Notes)
        self.Velocities_Embedding = Embedding(n_Velocities, self.d_Velocities)

    def call(self, x):
        chan = self.Channel_Embedding(x[0])
        note = self.Notes_Embedding(x[1])
        velo = self.Velocities_Embedding(x[2])
        
        # Scale to values between 0 and 1 ? (/time_range[1])
        time = tf.expand_dims(x[3],-1)
        return concatenate([chan,note,velo,time])

In [35]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, d_model, seq_length):
        super().__init__()
        self.d_model = d_model
        self.seq_length = seq_length
        self.pos_encoding = self.positional_encoding(seq_length, d_model)
        
    def positional_encoding(self, length, depth):
        depth = depth/2

        positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
        depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

        angle_rates = 1 / (10000**depths)         # (1, depth)
        angle_rads = positions * angle_rates      # (pos, depth)

        pos_encoding = np.concatenate([np.sin(angle_rads), np.cos(angle_rads)],axis=-1) 

        return tf.cast(pos_encoding, dtype=tf.float32)[tf.newaxis, :, :]

    def call(self, x):
        # This factor sets the relative scale of the embedding and positonal_encoding.
        # x *= tf.math.sqrt(tf.cast(self.d_model, tf.float64))
        return x + self.pos_encoding

In [37]:
class SelfAttentionBlock(tf.keras.layers.Layer):
    # Dropout ??
    def __init__(self, num_heads, d_model, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads, d_model, **kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

    def call(self, x):
        attn_output = self.mha(query=x,value=x,key=x,use_causal_mask = True)
        x = self.add([x, attn_output])
        x = self.layernorm(x)
        return x

In [39]:
class FeedForward(tf.keras.layers.Layer):
    def __init__(self, d_model, dff, dropout_rate=0.1):
        super().__init__()
        self.seq = tf.keras.Sequential([
          tf.keras.layers.Dense(dff, activation='relu'),
          tf.keras.layers.Dense(d_model),
          tf.keras.layers.Dropout(dropout_rate)
        ])
        self.add = tf.keras.layers.Add()
        self.layer_norm = tf.keras.layers.LayerNormalization()

    def call(self, x):
        x = self.add([x, self.seq(x)])
        x = self.layer_norm(x) 
        return x

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self,d_model,num_heads,dff,dropout_rate=0.1):
        super().__init__()
        self.self_attention_block = SelfAttentionBlock(num_heads,d_model)
        self.ffn = FeedForward(d_model, dff)

    def call(self, x):
        x = self.self_attention_block(x)
        x = self.ffn(x)
        x = self.dropout(x)
        return x

In [ ]:
class Transformer(tf.keras.layers.Layer):
    def __init__(self, n_Channels, n_Notes, n_Velocities, seq_length, d_model, n_layers, n_heads, dff, dropout_rate=0.1):
        super().__init__()
        self.embedding = MusicEmbedding(n_Channels, n_Notes, n_Velocities, d_model)
        self.pos_enc = PositionalEncoding(d_model, seq_length)

In [40]:
chan = np.array([[1,2,2,0]],dtype=int)
note = np.array([[0,54,78,39]],dtype=int)
velo = np.array([[127,0,32,49]],dtype=int)
time = np.array([[0.541,0.0236,1.754,0.0416]])

dim = 50
seq_length = 4
n_heads = 2

musEmb = MusicEmbedding(n_Channels, n_Notes, n_Velocities, dim)
posEnc = PositionalEncoding(dim, seq_length)
selAtt = SelfAttentionBlock(n_heads,dim)
fedFor = FeedForward(dim,4*dim)

x = musEmb.call([chan,note,velo,time])
y = posEnc.call(x)
z = selAtt.call(y)
o = fedFor.call(o)
print(o)

tf.Tensor(
[[[ 0.1498716  -0.85612124 -1.1648456  -1.1308112  -0.87213653
   -1.1369947  -0.7647081  -0.43588072  0.02077937 -0.97929984
   -1.3925036  -1.247667   -1.9367722   0.43962714 -0.81963825
   -0.95911884 -1.556759   -0.26435328 -0.23319872  0.09209898
   -0.08885338 -0.23668104 -1.6400095  -1.5634454  -1.3997375
    1.0864894  -0.2863794   0.92640597  1.3242077   1.0166073
    0.07095198  1.4390601   1.118162    0.57787865  0.35616288
    0.6363855   0.13609928  1.281493    0.5349418   0.44104514
    0.87509614  0.48874077  1.2036344   1.3760026   0.145619
    0.85943335  0.02686546  1.1155527   2.3972974   0.82940584]
  [ 1.4470997   0.22542562 -0.4687415  -0.56804615 -0.5025308
   -1.1563098  -1.0686269  -0.5042613   0.03162462 -0.71868485
   -1.224994   -1.6646316  -1.5910096   0.09239317 -1.4874991
   -1.3530526  -1.5160458  -0.6443248  -0.68528724 -0.1939443
   -0.1922605  -0.45660013 -1.9647781  -1.1683023  -1.5229385
    0.86588293 -0.1301964   0.5302467   1.4896765  